# Stochastic Neighborhood Embedding
<!-- 11-3 -->


***Introduction***

확률적 근방 임베딩 (Stochastic Neighborhood Embedding) 방법은 MDS와 거의 유사한 idea인 원래공간의 거리를 축소된 공간에서 보존하는 방법을 제안하고 있습니다. 

MDS와 SNE의 차이점은 다음과 같습니다.
- 원본 공간, 축소된 공간에서 데이터간의 거리를 정의하는 방법이 다릅니다. 사실 SNE에서는 비유사도로 볼 수 있는 거리 대신 확률로 정의한 유사도 측도를 활용합니다. 
- 원본공간에서 만들어진 유사도행렬과 축소된 공간에서 만들어진 유사도행렬간의 차이를 Frobenius norm이 아니라 KL-divergence로 정의합니다. 
- 축소된 공간의 데이터 표현형 직접 찾습니다. 

***conditinoal probablity and similarity***

원본 데이터 $x_i\in \mathbb{R}^d$ for $i = 1, \cdots, n $이  있다고 가정해보겠습니다. 우리는 데이터 $i$를 기준으로 나머지 데이터들의 유사도를 데이터간의 유클리디안 거리에 기반하여 정하려고 합니다. 다음 식을 보겠습니다. 

$$p_{j|i} = \frac{\exp( - \|x_i- x_j\|^2/\sigma_i^2)}{\sum_{k \neq i} \exp( - \|x_i- x_k\|^2/\sigma_i^2) }$$

$p_{j|i}$값은 $d(x_i, x_j) = \|x_i- x_j\|$의 크기가 커질 수록 작어지게 됩니다. 반대로 데이터간의 유클리디안 거리가 가까워질 수록  $p_{j|i}$값은 커질 것입니다. 그래서 $p_{j|i}$는 원본 공간에서 데이터의 거리에 기반한 유사도측도라고 볼 수 있습니다. 

그러면 축소된 공간에서 같은 데이터 $y_i\in \mathbb{R}^q$ for $i = 1, \cdots,n$ 를 생각해봅시다.  이 데이터들 역시 축소된 공간에서 유클리디안 거리에 기반하여 유사도를 정의할 수 있을 것입니다. 이를 $q_{j|i} $라고 하겠습니다.


$$q_{j|i} = \frac{\exp( - \|y_i- y_j\|^2)}{\sum_{k \neq i} \exp( - \|y_i- y_k\|^2) }$$
(차원 축소가 잘 되서 축소된 데이터의 산포가 유사하게 맞춰짐을 가정하여 조건부확률에 $\sigma_i^2$항이 없습니다. 결국 이런 방법으로 데이터가 잘 임베딩 된 $y_i$들을 찾고자 하기 때문에 $\sigma_i^2$항을 모두 동일하게 두었다고 이해해도 됩니다. )

그러면 우리는 다음과 같은 것을 기대할 것입니다. 원본 데이터 공간에서 $p_{j|i}$와 $q_{j|i}$가 유사하기를 원하겠지요. 이것은 MDS에서 원본 공간에 거리행렬과 축소된 공간에서 거리행렬이 유사하기를 기대하는 것과 동일한 아이디어네요. 

***Loss function of $q$ for a given $p$***

만약 $q_{j|i}$가 $p_{j|i}$ for all $j$ and for all $i$에 대해서 유사하다면 어떤 결론을 내릴 수 있을까요? "$q_{j|i}$들을 만들어주는 $y_i$들이 $\mathbb{R}^q$위에서 잘 임베딩되었구나"는 결론을 내릴 수 있겠지요? 

그러면 우리의 원하는 바 $q_{j|i}$와 $p_{j|i}$가 유사하게 만들기 위한 $y_i$를 찾는 이 목적을 어떻게 달성할 수 있을까요? 여기서 **KL-divergence**라는  개념을 도입합니다.  **KL-divergence**는 두 분포간의 비유사도를 재는 측도입니다. 어떤 분포 $P$와 어떤 분포 $Q$가 있다고 합시다. 이 두 분포가 얼마나 비슷하지 않은지를 재는 것으로
아래와 같이 표시합니다. 
$$KL(P||Q)$$ 
의미는 분포 $P$가 $Q$로 부터 떨어진 정도에 해당하며 $P=Q$ (분포의 의미에서 )일때 $KL(P||Q) = 0$입니다. 
계산은 이산형분포를 가정할 때 같은 결과값에 대해 $P$와 $Q$의 확률을 $p_i$와 $q_i$로 표시한다면
$$KL(P||Q) = \sum_{i} p_i \log \frac{ p_i}{q_i}$$
로 계산합니다. 

우리의 문제로 돌아가서, 조건부 분포  $q_{\cdot|i}$로 부터 p_{\cdot|i}$의 KL-divergence는 
$$KL(p_{\cdot|i}||q_{\cdot|i}) = \sum_{j} p_{j|i} \log \frac{ p_{j|i}}{q_{j|i}}$$
로 주어집니다. 

즉 우리는 데이터 전체에서 KL-divergence를 줄이려면 
$$KL(P||Q) = \sum_{i=1}^bKL(p_{\cdot|i}||q_{\cdot|i})$$
를 최소화하면 될 것입니다. 

***finding $Q$ via $y$***

$p_{\cdot|i}$들이 주어져 있다고 가정하고 $\sum_{i=1}^bKL(p_{\cdot|i}||q_{\cdot|i})$를 함수로 본다면 입력값이 무엇인지 확인해봅시다. 

$$q_{j|i} = \frac{\exp( - \|y_i- y_j\|^2)}{\sum_{k \neq i} \exp( - \|y_i- y_k\|^2) }$$
정의에 의해서 $q_{j|i}$는 사실 $y_1, \cdots, y_n$의 함수입니다. 이를 조금 명확하게 하기 위해서 

$q_{j|i}$를 $g_{j|i}(y)$라고 표시하겠습니다. 그러면, 

\begin{aligned}
KL(P||Q) &=& \sum_{i=1}^n \sum_{j=1}^n p_{j|i}\log p_{j|i} - p_{j|i}\log q_{j|i}\\
         &=& - \sum_{i=1}^n \sum_{j=1}^n p_{j|i}\log g_{j|i}(y) + \text{Constant}
\end{aligned}

이 함수를 최소화하는 $y$를 찾는 것이 SNE입니다. 

In [2]:

a = 1